Upload files to S3 bucket for storage.

In [2]:
import pandas as pd

dates1 = pd.date_range(start='2015-12-01', end='2015-12-02', freq='H', closed='left')
dates2 = pd.date_range(start='2016-04-15', end='2016-04-16', freq='H', closed='left')
dates3 = pd.date_range(start='2016-10-30', end='2016-10-31', freq='H', closed='left')

dates = dates1.union(dates2).union(dates3)

datess = []
for i in range(len(dates)):
    datess.append(str(dates[i]).replace(" ", "_") + "_UTC")
dates = datess

In [3]:
import boto3
import pandas as pd
from sagemaker import get_execution_role
import tqdm

role = get_execution_role()

for date in tqdm.tqdm(dates):
    obj = "tweets_spanish_" + date + ".json"
    obj_loc = "../../data/" + obj
    boto3.Session().resource('s3').Bucket("mt5599").Object("tweets/" + obj).upload_file(obj_loc)

100%|██████████| 72/72 [00:45<00:00,  1.59it/s]


Reading in raw dataframes so they can be combined.

In [6]:
import boto3
import pandas as pd
from sagemaker import get_execution_role
import tqdm

role = get_execution_role()

files = []
DataFrameName = []
for date in tqdm.tqdm(dates):
    files.append("s3://mt5599/tweets/tweets_spanish_" + str(date) + ".json")
    DataFrameName.append("tweets_spanish_" + str(date))

dfs = {}

for name, file in tqdm.tqdm(zip(DataFrameName, files)):
    dfs[name] = pd.read_json(file)

100%|██████████| 72/72 [00:00<00:00, 329683.28it/s]
72it [02:28,  2.06s/it]


Combining handles dataset and uploading it to S3 bucket.

In [7]:
df = pd.concat(dfs.values(), ignore_index=True)
df

,id,DateTime,tweet_content,cashtags,coordinates,hashtags,lang,like_count,media,mentioned_users,...,user_profile_image_url,user_profile_banner_url,user_label,place_country,place_country_code,place_full_name,place_name,place_type,coordinates_longitude,coordinates_latitude
0,671493915590057984,2015-12-01 00:59:59+00:00,mi fin de semana estuvo cabron,None,"Coordinates(longitude=-67.054105, latitude=18....",None,es,1,None,None,...,https://pbs.twimg.com/profile_images/160177785...,https://pbs.twimg.com/profile_banners/16040915...,None,United States,US,"Bejucos, Puerto Rico",Bejucos,city,-67.054105,18.481387
1,671493915497885696,2015-12-01 00:59:59+00:00,Ya me descarge el app😁 ahora mati no me va a p...,None,"Coordinates(longitude=-58.346384, latitude=-34...",None,es,1,None,None,...,https://pbs.twimg.com/profile_images/153241012...,https://pbs.twimg.com/profile_banners/22690117...,None,Argentina,AR,"Quilmes, Argentina",Quilmes,city,-58.346384,-34.801165
2,671493915342688256,2015-12-01 00:59:59+00:00,@gabyespinony Dejame buscar y te aviso !!!,None,"Coordinates(longitude=-73.3540725, latitude=0....",None,es,0,None,"[User(username='gabyespinony', id=2769738383, ...",...,https://pbs.twimg.com/profile_images/152098656...,None,None,Venezuela,VE,Venezuela,Venezuela,country,-73.354073,0.648837
3,671493914801643520,2015-12-01 00:59:59+00:00,Sin dudas la mejor gestión,None,"Coordinates(longitude=-64.3195, latitude=-31.5...",None,es,0,None,None,...,https://pbs.twimg.com/profile_images/160430215...,https://pbs.twimg.com/profile_banners/43886449...,None,Argentina,AR,"Córdoba, Argentina",Córdoba,city,-64.319500,-31.523576
4,671493914176724992,2015-12-01 00:59:59+00:00,Elbo Lita jajajaja que cldo!,None,"Coordinates(longitude=-64.389839, latitude=-33...",None,es,0,None,None,...,https://pbs.twimg.com/profile_images/140897405...,https://pbs.twimg.com/profile_banners/22708841...,None,Argentina,AR,"Río Cuarto, Argentina",Río Cuarto,city,-64.389839,-33.159419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1655961,792863655096442880,2016-10-30 23:00:00+00:00,"@CFKArgentina con las propiedades de Báez,cua...",None,"Coordinates(longitude=-63.2919697, latitude=-3...",None,es,0,None,"[User(username='CFKArgentina', id=138814032, d...",...,https://pbs.twimg.com/profile_images/149981312...,https://pbs.twimg.com/profile_banners/76809874...,None,Argentina,AR,"General San Martín, Argentina",General San Martín,city,-63.291970,-32.432099
1655962,792863654274367488,2016-10-30 23:00:00+00:00,"Temperatura: 22,4°C S.Térmica 22,4°C PR 12,7°C...",None,"Coordinates(longitude=-58.54916667, latitude=-...",None,es,1,None,None,...,https://pbs.twimg.com/profile_images/160486963...,https://pbs.twimg.com/profile_banners/47917531...,None,Argentina,AR,"Ciudad del Libertador General San Martín, Arge...",Ciudad del Libertador General San Martín,city,-58.549167,-34.535278
1655963,792863654089723904,2016-10-30 23:00:00+00:00,"que ondon Ramón con el camarón, que hongo joro...",None,"Coordinates(longitude=-97.43391558, latitude=3...",None,es,0,None,None,...,https://pbs.twimg.com/profile_images/839951285...,https://pbs.twimg.com/profile_banners/80163931...,None,United States,US,"Texas, USA",Texas,admin,-97.433916,30.346318
1655964,792863653401722880,2016-10-30 23:00:00+00:00,Extrañaba esto ☀ @ Nacimiento. Región Bio Bio ...,None,"Coordinates(longitude=-72.6704209, latitude=-3...",None,es,0,None,None,...,https://pbs.twimg.com/profile_images/760588784...,https://pbs.twimg.com/profile_banners/21230792...,None,Chile,CL,"Nacimiento, Chile",Nacimiento,city,-72.670421,-37.501660


Extracting all handles from Argentina.

In [10]:
df.to_json("../../data/handles_combined.json")

In [12]:
! pip install feather-format

Keyring is skipped due to an exception: 'keyring.backends'
  Using cached feather-format-0.4.1.tar.gz (3.2 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for feather-format: filename=feather_format-0.4.1-py3-none-any.whl size=2453 sha256=cb8f59798d5ee61ecc7e580075bbd78a411a8bc0a543a366b3679f82d36a20f2
  Stored in directory: /root/.cache/pip/wheels/6f/77/1d/0a36541cf0c2bb1c7bf319185290de60883f3edb8fccbe8a40
Successfully built feather-format


In [14]:
boto3.Session().resource('s3').Bucket("mt5599").Object("tweets/handles_combined.json").upload_file("../../data/handles_combined.json")

In [ ]:
# number of tweets gathered in spanish by country code (all geotagged)

df.place_country_code.value_counts(dropna=False)

In [ ]:
ar_usernames = df[df.place_country_code == "AR"]["username"].unique()
len(ar_usernames)

Saving handles to S3 bucket.

In [ ]:
with open("../../data/spanish_speakers_2016.txt", "w") as f:
    for handle in ar_usernames:
        f.write(f"{handle}\n")

boto3.Session().resource('s3').Bucket("mt5599").Object("tweets/spanish_speakers_2016.txt").upload_file("../../data/spanish_speakers_2016.txt")